# Cfr. Shift fit model in GCMs and Reanalyis

In [1]:
# necessary: netcdf4-python, regionmask, (xarray, geopandas), cartopy, openpyxl, statsmodels

import numpy as np
import pandas as pd
import os, glob, re 
import math
import xarray as xr
import geopandas as gpd
import regionmask as regionmask
import dask
import matplotlib.pyplot as plt
import netCDF4

%matplotlib inline

#plotting
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from scipy.stats import norm

# import my variables settings functions from other scripts
from settings_ana import *
from functions_ana import *
from utils_ana import *
from plotscript import *

# # import other packages/modules 
# sys.path.append(os.path.join(os.environ['VSC_DATA_VO_USER'],'demographics4climate/'))
# from population_demographics import * 

sys.path.append('../dist_cov/dist_cov/')
import distributions as distributions


In [2]:
model_years = calc_warming_periods_models(GCMs, dir_gmst_models, observed_warming_path, target_year=2023, method='ar6', match='closest', windowsize=0)[['year']]
model_years

,year
model,
CanESM5,2009
CNRM-CM6-1,2025
GFDL-ESM4,2035
IPSL-CM6A-LR,2012
MIROC6,2038
MRI-ESM2-0,2024
EC-Earth3,2017
UKESM1-0-LL,2022
MPI-ESM1-2-HR,2024


In [3]:
flags['models']='ISIMIP3b'
dirname = 'output_shift-fit' 

da_list = []

for i in range(len(GCMs)):
    GCM = GCMs[i]
    filepath = glob.glob(os.path.join('/data/brussel/vo/000/bvo00012/vsc10419/attr-hw/output', 
                                      f'output_shift-fit/forster2024/WBGT/ISIMIP3b/{GCM}/*_WBGT_params_shift_loc_mon_1901_2019.nc'))[0]
    da = xr.open_dataarray(filepath).expand_dims("dataset").assign_coords(dataset=("dataset", [GCM]))
    da_list.append(da)
    
    da_params = xr.concat(da_list, dim="dataset") #loc sigma long

In [4]:
da_params

<xarray.DataArray 'fit_params' (dataset: 10, month: 12, lat: 360, lon: 720,
                                params: 3)>
array([[[[[-2.56541812e+01, -6.38589957e-01,  6.29457830e+00],
          [-2.56556172e+01, -6.49277147e-01,  6.30100682e+00],
          [-2.56435135e+01, -7.27748910e-01,  6.29189939e+00],
          ...,
          [-2.56426722e+01, -6.57334627e-01,  6.29424479e+00],
          [-2.56223005e+01, -7.48470966e-01,  6.29212653e+00],
          [-2.56251386e+01, -7.56068780e-01,  6.29960224e+00]],

         [[-2.67648796e+01,  1.00129677e+00,  6.39906481e+00],
          [-2.67702948e+01,  9.89828158e-01,  6.39587941e+00],
          [-2.67707252e+01,  9.51983590e-01,  6.39608717e+00],
          ...,
          [-2.67519570e+01,  9.79090910e-01,  6.40879977e+00],
          [-2.67382140e+01,  9.33392878e-01,  6.39930316e+00],
          [-2.67445624e+01,  9.42521502e-01,  6.40316730e+00]],

         [[-2.83428253e+01,  3.48203052e+00,  6.65275342e+00],
          [-2.83506273e+01,  3.48942585e+00,  6.65116950e+00],
          [-2.83523067e+01,  3.48886859e+00,  6.66459866e+00],
          ...,
...
          ...,
          [-1.25149721e+01,  1.23320100e+00,  2.38042861e+00],
          [-1.25130006e+01,  1.23093237e+00,  2.37757624e+00],
          [-1.25082699e+01,  1.23301167e+00,  2.38289791e+00]],

         [[-1.32789391e+01,  2.01713073e+00,  2.45846855e+00],
          [-1.32777441e+01,  2.02095279e+00,  2.45432762e+00],
          [-1.32760997e+01,  2.01658703e+00,  2.45803290e+00],
          ...,
          [-1.32869108e+01,  2.01879346e+00,  2.45752252e+00],
          [-1.32832159e+01,  2.01835396e+00,  2.46192510e+00],
          [-1.32790949e+01,  2.01959288e+00,  2.45550381e+00]],

         [[-1.25098988e+01,  6.21610636e-01,  2.46446206e+00],
          [-1.25085391e+01,  6.16510415e-01,  2.46713660e+00],
          [-1.25097098e+01,  6.21607680e-01,  2.46482709e+00],
          ...,
          [-1.25124082e+01,  6.17155228e-01,  2.46926092e+00],
          [-1.25095892e+01,  6.19359487e-01,  2.46693010e+00],
          [-1.25107508e+01,  6.17303644e-01,  2.46839549e+00]]]]])
Coordinates:
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * params   (params) object 'b0' 'b1' 'sigma'
  * dataset  (dataset) <U13 'CanESM5' 'CNRM-CM6-1' ... 'CNRM-ESM2-1'

In [5]:
df_gmst_mod = merge_model_gmst(GCMs, dir_gmst_models)
df_gmst_mod_smo = df_gmst_mod.apply(lambda col: apply_lowess(col, df_gmst_mod.index, ntime=4))
df_gmst_mod_smo

,CanESM5,CNRM-CM6-1,GFDL-ESM4,IPSL-CM6A-LR,MIROC6,MRI-ESM2-0,EC-Earth3,UKESM1-0-LL,MPI-ESM1-2-HR,CNRM-ESM2-1
year,,,,,,,,,,
1850,0.081839,0.075265,-0.116804,-0.163789,-0.073311,0.073341,-0.082681,-0.062625,-0.005754,0.124842
1851,-0.027388,0.041693,-0.073870,-0.071802,-0.082864,0.061162,0.002303,-0.021973,0.078604,0.188433
1852,-0.087118,0.024950,-0.009374,-0.008409,-0.066935,0.019337,-0.004756,0.035597,0.199157,0.173887
1853,-0.054830,0.025412,-0.004595,0.037362,0.026333,-0.047268,-0.075239,0.064067,0.104657,0.095709
1854,-0.061384,0.024409,-0.085900,0.065928,0.191804,-0.040739,-0.128356,0.021006,0.010157,-0.000254
...,...,...,...,...,...,...,...,...,...,...
2096,6.125161,4.733954,3.435702,5.299199,3.125304,3.779781,4.628006,5.891278,3.618147,4.389936
2097,6.248846,4.811054,3.459183,5.345967,3.036804,3.785240,4.909858,5.991796,3.622728,4.476962
2098,6.367561,4.867603,3.484327,5.373768,3.152982,3.818553,4.805906,6.082378,3.681728,4.505526


In [6]:
da_gmst_mod_smo = xr.DataArray(df_gmst_mod_smo, dims=["year", "dataset"])

In [10]:
res = calc_nAHD_shift_fit(da_params, threshold=28, gmst_smo=da_gmst_mod_smo,year_pres=2023,GWI=1.3)

In [13]:
da_gmst_mod_smo.loc[2023] - 1.3

<xarray.DataArray (dataset: 10)>
array([ 0.70948764, -0.02117382, -0.23315513,  0.12589159, -0.48061348,
        0.01716026,  0.2984056 ,  0.17651716, -0.08496262, -0.16623883])
Coordinates:
    year     int64 2023
  * dataset  (dataset) object 'CanESM5' 'CNRM-CM6-1' ... 'CNRM-ESM2-1'

In [9]:
df_gmst_mod_smo.loc[2023].values

array([2.00948764, 1.27882618, 1.06684487, 1.42589159, 0.81938652,
       1.31716026, 1.5984056 , 1.47651716, 1.21503738, 1.13376117])